# 리뷰 크롤링

In [1]:
import selenium 
from selenium import webdriver
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import pickle
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import re

In [2]:
header={
    'Host': 'everytime.kr',
    'Connection': 'keep-alive',
    'Content-Length': '45',
    'Accept':'*/*',
    'Origin': 'https://everytime.kr',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://everytime.kr/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie': 'etsid=s%3AZDdSNuhS5AUdNj8pvHkGVqWM7HwX5QFI.RPVTNCOyipi1mrRYeV%2B7BfJ8DQa5Kal7bw%2BDtsQ4yKg; _ga=GA1.2.1837060412.1581574157; _gid=GA1.2.453030296.1581574157; _gat_gtag_UA_22022140_4=1'
}

In [3]:
df_sum = pd.read_pickle('lecture_processed.pickle')

In [4]:
# 리뷰 크롤링
crawl_codes = list(df_sum['링크'].apply(lambda x: re.sub('[^0-9]','',x)))

nm = []
pf =[]
rev = []
sc = [] 
details=[]

for i in tqdm_notebook(crawl_codes):
    
    response = requests.post(
        'https://api.everytime.kr/find/lecture/article/list',
        headers=header,
        data='school_id=38&limit_num=1000&lecture_id={}'.format(i)
    )
    reviews = bs(response.text,'xml')
    try :
        name = reviews.find('lecture')['name']
        prof = reviews.find('lecture')['professor']
    
        detail={
        'name' : name,
        'prof' : prof,
        'detail' : {
            '출석':reviews.find('details')['assessment_attendance'],
            '성적비율':reviews.find('details')['assessment_grade'],
            '과제':reviews.find('details')['assessment_homework'],
            '팀플':reviews.find('details')['assessment_team'],
            '시험횟수':reviews.find('details')['exam_times']
            }
        }
        details.append(detail)
        for review in (reviews.findAll('article')) :
            nm.append(name)
            pf.append(prof)
            rev.append(review['text'])
            sc.append(review['rate'])
    except :
        pass

In [5]:
with open('details.pickle','wb') as f:
    pickle.dump(details,f)

In [6]:
# DF로 전처리
review_df = pd.DataFrame([rev,sc,nm,pf]).T
review_df.columns=['review','score','name','prof']
text = review_df['review']

# 특수문자 제거, 공백 1개로 통일
text = text.apply(lambda x:re.sub('\n',' ',x))
text = text.apply(lambda x:re.sub('\r',' ',x))
text = text.apply(lambda x:re.sub('[^가-힣|A-z|0-9]',' ',x))
text = text.apply(lambda x:re.sub('[ ]+',' ',x))

review_df['review'] = text
review_df.to_pickle('review_0213.csv')